In [10]:
from bs4 import BeautifulSoup
import requests
from concurrent.futures import ThreadPoolExecutor
import re

titles = []
prices = []
ratings = []
stock = []
genere = []
amounts = []

# Create a session for connection pooling
session = requests.Session()#reusa la misma conexion tcp para ser mas rapido y eficiente

def scrape_page_books(i):
    target = session.get(f'https://books.toscrape.com/catalogue/page-{i}.html')
    target.encoding = "utf-8"
    soup = BeautifulSoup(target.text, "html.parser")#create the bs object and turn the unorganized text into a structured data tree

    page_data = []

    for book in soup.find_all("article", class_="product_pod"):#intera sobre el contenedor de los libros
        title = book.h3.a["title"].split(":", 1)[0].strip()
        price = book.find("p", class_="price_color").text
        rating = book.find("p", class_="star-rating")["class"][-1]
        availibility = book.find("p", class_="instock availability").text.strip()

        book_url = "https://books.toscrape.com/catalogue/" + book.h3.a["href"]
        
        page_data.append((title, price, rating, availibility, book_url))

    return page_data

#function to get data form the details page
def scrape_book_detail(book_url):
    detail = session.get(book_url)
    detail.encoding = "utf-8"
    soup_detail = BeautifulSoup(detail.text, "html.parser")

    genre = soup_detail.select("ul.breadcrumb li a")[2].text

    availability_text = soup_detail.find("p", class_="instock availability").text.strip()

    amount = int(re.search(r"\((\d+) available\)", availability_text).group(1))

    return genre, amount


all_books = []

with ThreadPoolExecutor(max_workers=10) as executor:
    futures = []

    for page_num in range(1, 52):
        futures.append(executor.submit(scrape_page_books, page_num))

    for future in futures:
        all_books.extend(future.result())



detail_results = []

with ThreadPoolExecutor(max_workers=20) as executor:
    futures = []

    for book in all_books:
        book_url = book[4]
        futures.append(executor.submit(scrape_book_detail, book_url))

    for future in futures:
        detail_results.append(future.result())



i = 0
for book in all_books:
    title = book[0]
    price = book[1]
    rating = book[2]
    availability = book[3]

    genre = detail_results[i][0]
    amount = detail_results[i][1]

    titles.append(title)
    prices.append(price)
    ratings.append(rating)
    stock.append(availability)
    amounts.append(amount)
    genere.append(genre)

    i += 1


print(f"Scraped {len(titles)} books")

Scraped 1000 books


In [11]:
import mysql.connector
import random

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password=""
)

cursor = conn.cursor()

cursor.execute("CREATE DATABASE IF NOT EXISTS library")
cursor.execute("USE library")


# Author table
cursor.execute("""
CREATE TABLE IF NOT EXISTS author (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(100)
)
""")
cursor.execute("""
CREATE TABLE IF NOT EXISTS books (
    id INT AUTO_INCREMENT PRIMARY KEY,
    title VARCHAR(255),
    price VARCHAR(50),
    rating VARCHAR(10),
    availibility VARCHAR(50),
    amount_available INT,
    genre VARCHAR(50)
)
""")

# Many-to-many relation
cursor.execute("""
CREATE TABLE IF NOT EXISTS book_author (
    book_id INT,
    author_id INT,
    PRIMARY KEY (book_id, author_id),
    FOREIGN KEY (book_id) REFERENCES books(id),
    FOREIGN KEY (author_id) REFERENCES author(id)
)
""")



conn.commit()
cursor.close()
conn.close()


In [12]:
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="library"
)
cursor = conn.cursor()

cursor.execute("SET FOREIGN_KEY_CHECKS = 0;")
cursor.execute("TRUNCATE TABLE book_author;")
cursor.execute("TRUNCATE TABLE books;")
cursor.execute("TRUNCATE TABLE author;")
cursor.execute("SET FOREIGN_KEY_CHECKS = 1;")

for t, p, r, s, a, g in zip(titles, prices, ratings, stock, amounts, genere):
    cursor.execute("""
        INSERT INTO books (title, price, rating, availibility, amount_available, genre)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, (t, p, r, s, a, g))
# Random authors
authors = [
    "Juan Perez",
    "Maria Lopez",
    "Carlos Gomez",
    "Ana Torres",
    "Luis Fernandez"
]

cursor.executemany(
    "INSERT INTO author (name) VALUES (%s)",
    [(a,) for a in authors]
)


cursor.execute("SELECT id FROM books")
book_ids = []

rows = cursor.fetchall()
for b in rows:
    book_ids.append(b[0])


cursor.execute("SELECT id FROM author")
author_ids = []

rows = cursor.fetchall()

for a in rows:
    author_ids.append(a[0])


# Assign 1 or 2 authors per book
relations = []
for book_id in book_ids:
    for author_id in random.sample(author_ids, random.choice([1, 2])):
        relations.append((book_id, author_id))

cursor.executemany(
    "INSERT INTO book_author (book_id, author_id) VALUES (%s, %s)",
    relations
)

conn.commit()
cursor.close()
conn.close()


In [ ]:
import mysql.connector
import time

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="library"
)

cursor = conn.cursor()

queries = [
    ("Books by genre tarvel",
    "SELECT title FROM books WHERE genre = 'poetry'"),

    ("Books with rating Three",
    "SELECT title,genre FROM books WHERE rating = 'Three'"),

    ("Books in stock ordered by amount",
    """
    SELECT title, amount_available, genre
    FROM books
    WHERE amount_available > 0
    ORDER BY amount_available DESC
    """),

    ("travel written  by carlos",
    """
    SELECT b.title, b.genre, a.name as authors
    FROM books b
    JOIN book_author ba ON b.id = ba.book_id
    JOIN author a ON a.id = ba.author_id
    WHERE b.genre = 'travel' AND a.name = 'Carlos Gomez'    

    """),

    ("Cheap 5-star books from each categorie",
    """
    SELECT title, price, genre
    FROM books
    WHERE CAST(SUBSTRING(price, 2) AS DECIMAL(10,2)) < 30.00
    AND rating = 'Five'
    GROUP BY genre
    ORDER BY price ASC
    """)
]

for title, q in queries:
    print("\n", title)
    print("-" * 40)

    start = time.perf_counter()
    cursor.execute(q)
    rows = cursor.fetchall()
    end = time.perf_counter()

    for row in rows:
        print(row)

    print(f"\nTime taken: {end - start:.6f} seconds")  # elapsed time

cursor.close()
conn.close()


In [5]:
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="library"
)
cursor = conn.cursor()

cursor.execute("CREATE INDEX IF NOT EXISTS idx_books_genre ON books(genre);")
cursor.execute("CREATE INDEX IF NOT EXISTS idx_books_rating ON books(rating);")
cursor.execute("CREATE INDEX IF NOT EXISTS idx_books_amount ON books(amount_available);")
cursor.execute("CREATE INDEX IF NOT EXISTS idx_author_name ON author(name);")

conn.commit()
cursor.close()
conn.close()